In [1]:
import pulp as p
import pandas as pd
import numpy as np
from numpy import random
import math
import json

In [2]:
def readJsonWithParams(filename = 'input.json'):
    with open(filename, 'r') as reader:
        return json.loads(reader.read().replace('\n',''))
params = readJsonWithParams()

In [3]:
def getIntervalOfAlpha(alpha_means, mu, optimist = True):
    if(mu <= 0 or mu > 1):
        raise Exception('Mu should be less then 1 and greater then 0')
    result = []
    for alpha_mean in alpha_means:
        a = alpha_mean * (1/mu - 1)**(1/2)
        val = 0
        if(optimist):
            val = alpha_mean + a
        else:
            val = alpha_mean - a
        result.append(int(val))
    return result

# getIntervalOfAlpha([120, 90], 0.8)

In [4]:
def getIntervalOfC(c_means, sigmas, mu, optimist = True):
    if(mu <= 0 or mu > 1):
        raise Exception('Mu should be less then 1 and greater then 0')
    result = []
    for i in range(len(c_means)):
        c_mean = c_means[i]
        a = (2*math.log(1/mu))**(1/2)*sigmas[i]
        val = 0
        if(optimist):
            val += c_mean + a
        else:
            val += c_mean - a
        result.append(val)
    return result

# getIntervalOfC([2,3], [1,1], 0.8, True)

In [5]:
def createCuttingMaps(alpha, t):
    d = dict((el,[]) for el in alpha )
    for i in range(len(alpha)):
        for j in range (alpha[i]//t[0] + 1):
            ost = (alpha[i] - j * t[0])
            for n in range (ost // t[1] + 1):
                if(n != 0 or j != 0):
                    d[alpha[i]].append([j,n])
    return d

# createCuttingMaps(alpha, t)

In [6]:
def solver(a, alpha, k, t, objective_function_max_number_sets = True, c = []):
    # карты раскроя
    d = createCuttingMaps(alpha, t)
    
    # план карт раскроя
    x_ij_j = dict((el,[]) for el in alpha )
    for i in range(len(alpha)):
        al = alpha[i]
        for j in range(len(d[al])):
            di = d[al][j]
            x_ij_j[al].append(p.LpVariable(f'x_{i+1}__{di[0]}_{di[1]}', lowBound=0, cat='Integer'))
    
    # создание задачи ЛП
    lp = p.LpProblem(f'Cutting_task_max_sets_{objective_function_max_number_sets}', p.LpMaximize)
    
    if(objective_function_max_number_sets):
        x = p.LpVariable("x", lowBound=0, cat='Integer')
        # целевая функция
        lp += x
        
        # добавление ограничений
        # ограничения на количество полуфабрикатов в партиях
        for i in range(len(a)):
            al = alpha[i]
            x_ij = p.LpAffineExpression()
            for el in x_ij_j[al]:
                x_ij += el
            lp += x_ij <= a[i]
#         print(k)
        # количество деталей в комплекте
        for detail in range(2):
            d_ij = p.LpAffineExpression()
            for i in range(len(a)):
                al = alpha[i]
                for j in range(len(x_ij_j[al])):
                    el = x_ij_j[al][j]
                    d_ij += el * d[al][j][detail]
            
            lp += (d_ij >= k[detail] * x)
            lp += (d_ij <= k[detail] * x)
    else: 
        # целевая функция
        x = p.LpVariable("x", lowBound=0)
        objective_func = p.LpAffineExpression()
        for detail in range (2):
            for i in range(len(a)):
                al = alpha[i]
                for j in range(len(d[al])):
                    objective_func += c[detail] * d[al][j][detail] * x_ij_j[al][j]
        lp+= x
        # добавление ограничений
        # ограничения на количество полуфабрикатов в партиях
        for i in range(len(a)):
            al = alpha[i]
            x_ij = p.LpAffineExpression()
            for el in x_ij_j[al]:
                x_ij += el
            lp += x_ij <= a[i]
        # x == objective_func
        lp += (objective_func >= x)
        lp += (objective_func <= x)
    
    # решение
    status = lp.solve()
#     print(f'Solver: {lp.solver} ')
    print(p.LpStatus[status])
    if(objective_function_max_number_sets):
        print(f'Solve: max number of sets = {p.value(x)}')
    else: 
        print(f'Solve: max profit from details = {round(p.value(x),2)}')
    print('Plan:')
    for al in alpha:
        for xiji in x_ij_j[al]:
            if(p.value(xiji)!=0):
                print(f'\t{xiji} = {int(p.value(xiji))}')
    waste = 0;
    for detail in range(2):
        for i in range(len(alpha)):
            al = alpha[i]
            for j in range(len(d[al])):
                d_ = d[al][j][detail]
                x_ = p.value(x_ij_j[al][j])
#                 print(f'Waste {detail} {i} {j} = {al - t[detail] * d_ij * x_}')
                waste +=  - t[detail] * d_ * x_
    for i in range(len(a)):
        waste += a[i] * alpha[i]
        
    print(f'\nWaste = {waste}')
    
#     print(lp)
                
        

# solver(a, alpha, k, t)

In [7]:
# начальные значения с максимизацией количества комплектов
a = params['a']
alpha = params['alpha']
k = params['k']
t = params['t']

In [8]:
# решение целочисленной задачи раскроя 
solver(a, alpha, k, t, True)

Optimal
Solve: max number of sets = 29.0
Plan:
	x_1__2_4 = 16
	x_1__5_0 = 1
	x_1__5_2 = 1
	x_1__7_1 = 26
	x_2__1_3 = 37

Waste = 445.0


In [9]:
# начальные значения для задач с нечеткими числами 
alpha_means = params['alpha_means']
c_means = params['c_means']
sigmas = params['sigmas']
mu = params['mu']

In [10]:
alpha_optimist = getIntervalOfAlpha(alpha_means, mu, True)
c_optimist = getIntervalOfC(c_means, sigmas, mu, True)
print(f'alpha_optimist={alpha_optimist}')
print(f'c_optimist={c_optimist}')

alpha_optimist=[180, 135]
c_optimist=[2.6680472308365775, 3.6680472308365775]


In [11]:
alpha_pessimist = getIntervalOfAlpha(alpha_means, mu, False)
c_pessimist = getIntervalOfC(c_means, sigmas, mu, False)
print(f'alpha_pessimist={alpha_pessimist}')
print(f'c_pessimist={c_pessimist}')

alpha_pessimist=[60, 45]
c_pessimist=[1.3319527691634225, 2.3319527691634225]


In [12]:
# решение целочисленной задачи раскроя с нечеткими числами 
# (максимизация числа комплектов, оптимист)  
solver(a, alpha_optimist, k, t, True)

Optimal
Solve: max number of sets = 43.0
Plan:
	x_1__0_1 = 1
	x_1__4_5 = 22
	x_1__9_0 = 1
	x_1__12_0 = 21
	x_2__1_5 = 38

Waste = 201.0


In [13]:
# решение целочисленной задачи раскроя с нечеткими числами 
# (максимизация числа комплектов, пессимист) 
solver(a, alpha_pessimist, k, t, True)

Optimal
Solve: max number of sets = 12.0
Plan:
	x_1__0_2 = 42
	x_2__3_0 = 36

Waste = 774.0


In [14]:
# решение целочисленной задачи раскроя с нечеткими числами 
# (максимизация прибыли от деталей, оптимист) 
solver(a, alpha_optimist, k, t, False, c_optimist)

Optimal
Solve: max profit from details = 2353.22
Plan:
	x_1__12_0 = 45
	x_2__9_0 = 38

Waste = 0.0


In [15]:
# решение целочисленной задачи раскроя с нечеткими числами 
# (максимизация прибыли от деталей, пессимист) 
solver(a, alpha_pessimist, k, t, False, c_pessimist)

Optimal
Solve: max profit from details = 391.59
Plan:
	x_1__4_0 = 45
	x_2__3_0 = 38

Waste = 0.0


In [16]:
solver(a, [120, 72], k, t, False, c_means)

Optimal
Solve: max profit from details = 1062.0
Plan:
	x_1__8_0 = 45
	x_2__3_1 = 38

Waste = 114.0


In [17]:
solver(a, [120, 72], k, t, True)

Optimal
Solve: max number of sets = 26.0
Plan:
	x_1__0_5 = 13
	x_1__2_3 = 1
	x_1__8_0 = 29
	x_2__0_3 = 38

Waste = 258.0


In [18]:
# a = [random.randint(100000,200000) for x in range(2)]
# alpha = [random.randint(1000,10000) for x in range(2)]
# k = [random.randint(1,10) for x in range(2)]
# t = [random.randint(5,30) for x in range(2)]
# c = [random.randint(10,30) for x in range(2)]
# print(f'a = {a}')
# print(f'alpha = {alpha}')
# print(f'k = {k}')
# print(f't = {t}')
# print(f'c = {c}')

In [19]:
# максимизация количества наборов при случайных параметрах
# solver(a, alpha, k, t, True)

In [20]:
# максимизация прибили при случайных параметрах
# solver(a, alpha, k, t, False, c)

In [21]:
a = [random.randint(10,100) for x in range(2)]
alpha = [random.randint(50,200) for x in range(2)]
k = [random.randint(1,10) for x in range(2)]
t = [random.randint(5,30) for x in range(2)]
c = [random.randint(10,30) for x in range(2)]
sigmas = random.rand(2) + 1
mu = round(random.rand()/2,2) + 0.5
print(f'a = {a}')
print(f'alpha = {alpha}')
print(f'k = {k}')
print(f't = {t}')
print(f'c = {c}')
print(f'sigmas = {sigmas}')
print(f'mu = {mu}')

a = [21, 14]
alpha = [176, 143]
k = [3, 5]
t = [7, 11]
c = [10, 12]
sigmas = [1.46023436 1.31027053]
mu = 0.66


In [22]:
alpha_optimist = getIntervalOfAlpha(alpha, mu, True)
c_optimist = getIntervalOfC(c, sigmas, mu, True)
print(f'alpha_optimist={alpha_optimist}')
print(f'c_optimist={c_optimist}')

alpha_optimist=[302, 245]
c_optimist=[11.331162724259038, 13.194454350147161]


In [23]:
alpha_pessimist = getIntervalOfAlpha(alpha, mu, False)
c_pessimist = getIntervalOfC(c, sigmas, mu, False)
print(f'alpha_pessimist={alpha_pessimist}')
print(f'c_pessimist={c_pessimist}')

alpha_pessimist=[49, 40]
c_pessimist=[8.668837275740962, 10.805545649852839]


In [24]:
# решение целочисленной задачи раскроя с нечеткими числами  
# при случайных параметрах
# (максимизация числа комплектов, оптимист)  
solver(a, alpha_optimist, k, t, True)

Optimal
Solve: max number of sets = 128.0
Plan:
	x_1__0_25 = 1
	x_1__5_24 = 1
	x_1__7_23 = 3
	x_1__18_16 = 15
	x_1__29_9 = 1
	x_2__2_21 = 13
	x_2__33_0 = 1

Waste = 44.0


In [25]:
# решение целочисленной задачи раскроя с нечеткими числами  
# при случайных параметрах
# (максимизация числа комплектов, пессимист) 
solver(a, alpha_pessimist, k, t, True)

Optimal
Solve: max number of sets = 20.0
Plan:
	x_1__1_1 = 1
	x_1__2_3 = 19
	x_1__7_0 = 1
	x_2__1_3 = 14

Waste = 69.0


In [26]:
# решение целочисленной задачи раскроя с нечеткими числами 
# при случайных параметрах
# (максимизация прибыли от деталей, оптимист) 
solver(a, alpha_optimist, k, t, False, c_optimist)

Optimal
Solve: max profit from details = 15784.31
Plan:
	x_1__43_0 = 21
	x_2__35_0 = 14

Waste = 21.0


In [27]:
# решение целочисленной задачи раскроя с нечеткими числами 
# при случайных параметрах
# (максимизация прибыли от деталей, пессимист) 
solver(a, alpha_pessimist, k, t, False, c_pessimist)


Optimal
Solve: max profit from details = 1911.05
Plan:
	x_1__7_0 = 21
	x_2__4_1 = 14

Waste = 14.0
